# Trade Execution Workbook

**Workflow:**
1. Connect to IB Gateway and pull live positions
2. Generate trade recommendations (with cash reserve enforcement)
3. Review the trade plan and add instructions
4. Send instructions to Claude for interpretation
5. Execute trades on IB

---

In [ ]:
import csv
import json
import os
import shutil
import sys
from datetime import datetime
from pathlib import Path

import nest_asyncio
import pandas as pd
from IPython.display import display, HTML, Markdown

nest_asyncio.apply()

# Paths
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))
DATA_DIR = PROJECT_ROOT / "data"
TRADING_DIR = Path.home() / "trading"
LIVE_DIR = TRADING_DIR / "live_portfolio"
SNAPSHOT_DIR = LIVE_DIR / "snapshots"
ARCHIVE_DIR = LIVE_DIR / "trade_plan_archive"
TRADE_PLAN_FILE = LIVE_DIR / "trade_plan.csv"
EXCEL_FILE = TRADING_DIR / "portfolio_tracking.xlsx"

# IB settings
IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 3  # Unique client ID for notebook

# Strategy settings
CASH_RESERVE = 70_000
TRAILING_STOP_PCT = 10
ENTRY_STOP_LOSS_PCT = 0.12

# Ensure directories exist
LIVE_DIR.mkdir(parents=True, exist_ok=True)
ARCHIVE_DIR.mkdir(parents=True, exist_ok=True)
SNAPSHOT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root:  {PROJECT_ROOT}")
print(f"Trading dir:   {TRADING_DIR}")
print(f"Trade plan:    {TRADE_PLAN_FILE}")
print(f"Cash reserve:  ${CASH_RESERVE:,}")

---
## Step 1: Connect to IB and Pull Live Data

In [ ]:
from ib_insync import IB, Stock, Order, MarketOrder, LimitOrder

ib = IB()
print(f"Connecting to IB Gateway ({IB_HOST}:{IB_PORT})...")
ib.connect(IB_HOST, IB_PORT, clientId=IB_CLIENT_ID, readonly=True, timeout=10)
accts = ib.managedAccounts()
print(f"Connected. Account: {accts[0]}")

In [ ]:
# Pull account summary
summary = {}
for av in ib.accountSummary():
    if av.currency == "USD":
        try:
            summary[av.tag] = float(av.value)
        except (ValueError, TypeError):
            pass

nlv = summary.get("NetLiquidation", 0)
cash = summary.get("TotalCashValue", 0)
pos_val = summary.get("GrossPositionValue", 0)

# Pull positions
ib_positions = [
    {"ticker": p.contract.symbol,
     "shares": float(p.position),
     "avg_cost": float(p.avgCost)}
    for p in ib.positions()
]

# Load strategy targets
target_path = LIVE_DIR / "target_portfolio_latest.csv"
targets = {}
if target_path.exists():
    tdf = pd.read_csv(target_path, index_col=0)
    targets = {t: float(r.iloc[0]) for t, r in tdf.iterrows()}

# Load factor scores
scores_path = DATA_DIR / "factor_scores_latest.parquet"
scores = pd.Series(dtype=float)
if scores_path.exists():
    scores = pd.read_parquet(scores_path).iloc[:, 0].sort_values(ascending=False)

# Get live prices
all_tickers = sorted(
    set(p["ticker"] for p in ib_positions) | set(targets.keys())
)
print(f"Fetching prices for {len(all_tickers)} tickers...")

ib.reqMarketDataType(3)  # Delayed (free)
prices = {}
qualified = []
for t in all_tickers:
    c = Stock(t, "SMART", "USD")
    try:
        ib.qualifyContracts(c)
        if c.conId:
            qualified.append(c)
    except Exception:
        pass

snaps = [(c.symbol, ib.reqMktData(c, snapshot=True)) for c in qualified]
ib.sleep(4)

for sym, td in snaps:
    for attr in ("last", "close", "bid", "ask"):
        v = getattr(td, attr, None)
        if v is not None and v == v and v > 0:
            prices[sym] = float(v)
            break
    ib.cancelMktData(td.contract)

ib.reqMarketDataType(1)
print(f"Got {len(prices)} prices.")

# Account overview
reserve_ok = cash >= CASH_RESERVE
deployable = max(0, cash - CASH_RESERVE)

display(Markdown(f"""
### Account Overview
| Metric | Value |
|--------|-------|
| Net Liquidation | ${nlv:,.2f} |
| Cash | ${cash:,.2f} |
| Positions Value | ${pos_val:,.2f} |
| Invested | {pos_val/nlv*100:.1f}% |
| **Cash Reserve** | **${CASH_RESERVE:,}** |
| **Deployable Cash** | **${deployable:,.2f}** |
| Reserve Status | {'OK' if reserve_ok else 'BELOW MINIMUM'} |
"""))

---
## Step 2: Build Positions & Generate Trade Recommendations

In [ ]:
# Build position data
pos_data = []
for pos in ib_positions:
    t = pos["ticker"]
    if pos["shares"] == 0:
        continue
    price = prices.get(t, pos["avg_cost"])
    mv = pos["shares"] * price
    cb = pos["shares"] * pos["avg_cost"]
    pnl = mv - cb
    pnl_pct = pnl / cb if cb != 0 else 0
    tw = targets.get(t, 0)
    aw = mv / nlv if nlv else 0
    score = scores.get(t) if t in scores.index else None

    pos_data.append({
        "ticker": t, "shares": pos["shares"],
        "avg_cost": pos["avg_cost"], "price": price,
        "mkt_value": mv, "pnl": pnl, "pnl_pct": pnl_pct,
        "target_w": tw, "actual_w": aw,
        "drift": aw - tw,
        "score": score,
        "stop": pos["avg_cost"] * (1 - ENTRY_STOP_LOSS_PCT),
        "in_target": t in targets,
    })

pos_data.sort(key=lambda x: (-x["target_w"], -x["mkt_value"]))

# Display positions
pos_df = pd.DataFrame(pos_data)
display_cols = [
    "ticker", "shares", "avg_cost", "price",
    "mkt_value", "pnl", "pnl_pct", "target_w",
    "actual_w", "drift", "in_target",
]
styled = pos_df[display_cols].style.format({
    "avg_cost": "${:.2f}", "price": "${:.2f}",
    "mkt_value": "${:,.0f}", "pnl": "${:+,.0f}",
    "pnl_pct": "{:+.1%}", "target_w": "{:.1%}",
    "actual_w": "{:.1%}", "drift": "{:+.1%}",
}).applymap(
    lambda v: "color: green" if isinstance(v, (int, float)) and v > 0 else
              "color: red" if isinstance(v, (int, float)) and v < 0 else "",
    subset=["pnl", "pnl_pct", "drift"]
)
display(Markdown("### Current Positions"))
display(styled)

total_pnl = sum(p["pnl"] for p in pos_data)
winners = sum(1 for p in pos_data if p["pnl"] > 0)
print(f"\nTotal P&L: ${total_pnl:+,.0f}  |  "
      f"Winners: {winners}/{len(pos_data)}")

In [ ]:
# Generate trade recommendations with cash reserve enforcement
ib_map = {p["ticker"]: p for p in pos_data}
trades = []
special = {"IBKR"}

# 1. Sell non-target positions
for p in pos_data:
    if not p["in_target"] and p["ticker"] not in special:
        if p["shares"] > 0:
            trades.append({
                "action": "SELL", "ticker": p["ticker"],
                "shares": int(abs(p["shares"])),
                "price": round(p["price"], 2),
                "est_value": round(abs(p["mkt_value"]), 2),
                "reason": "Not in strategy target",
            })
        elif p["shares"] < 0:
            trades.append({
                "action": "BUY_TO_COVER", "ticker": p["ticker"],
                "shares": int(abs(p["shares"])),
                "price": round(p["price"], 2),
                "est_value": round(abs(p["mkt_value"]), 2),
                "reason": "Close short position",
            })

# Calculate available cash for buys
sell_proceeds = sum(t["est_value"] for t in trades if t["action"] == "SELL")
cover_cost = sum(t["est_value"] for t in trades if t["action"] == "BUY_TO_COVER")
available = max(0, cash + sell_proceeds - cover_cost - CASH_RESERVE)
print(f"Available for buys: ${available:,.0f}  "
      f"(cash ${cash:,.0f} + sells ${sell_proceeds:,.0f} "
      f"- covers ${cover_cost:,.0f} - reserve ${CASH_RESERVE:,})")

# 2. Buy candidates
buy_candidates = []

# Missing targets
for ticker, tw in targets.items():
    if ticker not in ib_map:
        price = prices.get(ticker)
        if price and price > 0:
            sh = int(tw * nlv / price)
            if sh > 0:
                buy_candidates.append({
                    "action": "BUY", "ticker": ticker,
                    "shares": sh, "price": round(price, 2),
                    "est_value": round(sh * price, 2),
                    "reason": f"New position (target {tw*100:.0f}%)",
                })

# Rebalance drifted (>3%)
for p in pos_data:
    if p["target_w"] > 0 and abs(p["drift"]) > 0.03:
        diff = p["target_w"] * nlv - p["mkt_value"]
        ds = int(abs(diff) / p["price"])
        if ds > 0:
            if diff > 0:
                buy_candidates.append({
                    "action": "BUY", "ticker": p["ticker"],
                    "shares": ds, "price": round(p["price"], 2),
                    "est_value": round(ds * p["price"], 2),
                    "reason": f"Rebalance (drift {p['drift']*100:+.1f}%)",
                })
            else:
                trades.append({
                    "action": "SELL", "ticker": p["ticker"],
                    "shares": ds, "price": round(p["price"], 2),
                    "est_value": round(ds * p["price"], 2),
                    "reason": f"Rebalance (drift {p['drift']*100:+.1f}%)",
                })

# Apply cash reserve cap
running_spend = 0
for bc in buy_candidates:
    if running_spend + bc["est_value"] <= available:
        trades.append(bc)
        running_spend += bc["est_value"]
    else:
        remaining = available - running_spend
        if remaining > bc["price"]:
            reduced = int(remaining / bc["price"])
            if reduced > 0:
                bc["shares"] = reduced
                bc["est_value"] = round(reduced * bc["price"], 2)
                bc["reason"] += f" [capped by ${CASH_RESERVE:,} reserve]"
                trades.append(bc)
                running_spend += bc["est_value"]

trades.sort(key=lambda t: (0 if "SELL" in t["action"] else 1, t["ticker"]))

# Add instruction column
for t in trades:
    t["instruction"] = "APPROVE"

trade_df = pd.DataFrame(trades)

total_buys = sum(t["est_value"] for t in trades if "BUY" in t["action"])
total_sells = sum(t["est_value"] for t in trades if t["action"] == "SELL")
cash_after = cash + total_sells - total_buys

display(Markdown(f"### Trade Recommendations ({len(trades)} trades)"))
display(trade_df)
print(f"\nTotal buys:  ${total_buys:,.0f}")
print(f"Total sells: ${total_sells:,.0f}")
print(f"Cash after:  ${cash_after:,.0f}  (reserve: ${CASH_RESERVE:,})")

---
## Step 3: Review & Edit Instructions

Edit the `instruction` column below. Options:
- `APPROVE` — execute as-is
- `SKIP` — do not execute
- **Plain English** — Claude will interpret it, e.g.:
  - `reduce to 30 shares`
  - `change to limit order at $85`
  - `buy 100 shares instead`
  - `only if price drops below $40`

Edit the dictionary below, then run the cell.

In [ ]:
# ============================================================
# EDIT YOUR INSTRUCTIONS HERE
# ============================================================
# Format: "TICKER": "instruction"
# Only include tickers you want to change from APPROVE.
# Everything not listed stays as APPROVE.
#
# Examples:
#   "BND": "SKIP",
#   "DIM": "reduce to 30 shares",
#   "EWU": "change to limit order at $45",
#   "IDEV": "buy 50 shares instead",

custom_instructions = {
    # "BND": "SKIP",
}

# ============================================================
# Apply instructions to trade plan
# ============================================================
for t in trades:
    if t["ticker"] in custom_instructions:
        t["instruction"] = custom_instructions[t["ticker"]]

trade_df = pd.DataFrame(trades)
display(Markdown("### Updated Trade Plan"))
display(trade_df)

---
## Step 4: Archive & Save Trade Plan

In [ ]:
# Archive previous trade plan if it exists
if TRADE_PLAN_FILE.exists():
    mtime = datetime.fromtimestamp(TRADE_PLAN_FILE.stat().st_mtime)
    archive_name = f"trade_plan_{mtime.strftime('%Y%m%d_%H%M%S')}.csv"
    dest = ARCHIVE_DIR / archive_name
    shutil.copy2(TRADE_PLAN_FILE, dest)
    print(f"Archived previous plan: {dest}")

# Write new trade plan
ts = datetime.now().strftime("%Y-%m-%d %H:%M")
with open(TRADE_PLAN_FILE, "w", newline="") as f:
    f.write(f"# TRADE PLAN - Generated {ts}\n")
    f.write("#\n")
    writer = csv.DictWriter(
        f,
        fieldnames=[
            "action", "ticker", "shares", "price",
            "est_value", "reason", "instruction",
        ],
    )
    writer.writeheader()
    for t in trades:
        writer.writerow(t)

print(f"Trade plan saved: {TRADE_PLAN_FILE}")
print(f"{len(trades)} trade(s) ready.")

# Show archive contents
archives = sorted(ARCHIVE_DIR.glob("trade_plan_*.csv"))
if archives:
    print(f"\nArchive ({len(archives)} previous plans):")
    for a in archives[-5:]:
        print(f"  {a.name}")
    if len(archives) > 5:
        print(f"  ... and {len(archives) - 5} more")

---
## Step 5: Interpret Instructions with Claude

If you have custom (non-APPROVE/SKIP) instructions, this cell sends them to Claude for interpretation. Set `ANTHROPIC_API_KEY` in your environment.

In [ ]:
def interpret_trades(trades):
    """Interpret trade instructions, using Claude for custom ones."""
    result = []
    custom = [
        t for t in trades
        if t["instruction"].upper() not in ("APPROVE", "SKIP", "")
    ]

    # Claude interpretation for custom instructions
    claude_map = {}
    if custom:
        api_key = os.environ.get("ANTHROPIC_API_KEY")
        if api_key:
            try:
                import anthropic
                client = anthropic.Anthropic(api_key=api_key)

                trade_lines = []
                for t in custom:
                    trade_lines.append(
                        f"  - {t['action']} {t['shares']} shares of "
                        f"{t['ticker']} @ ${t['price']:.2f} | "
                        f"Reason: {t['reason']} | "
                        f'User instruction: "{t["instruction"]}"'
                    )

                prompt = f"""You are a trade execution assistant. Interpret each user instruction into concrete trade parameters.

Return a JSON array of trade objects with these fields:
- "ticker": string
- "action": "BUY" | "SELL" | "BUY_TO_COVER" | "SKIP"
- "order_type": "MARKET" | "LIMIT" | "STOP"
- "shares": integer
- "limit_price": number or null
- "stop_price": number or null
- "note": string (brief explanation)

Trades:
{chr(10).join(trade_lines)}

Rules:
- If ambiguous, default to SKIP with explanation
- Default to MARKET if no order type specified
Return ONLY the JSON array."""

                print("Sending to Claude...")
                msg = client.messages.create(
                    model="claude-sonnet-4-5-20250929",
                    max_tokens=1024,
                    messages=[{"role": "user", "content": prompt}],
                )
                text = msg.content[0].text.strip()
                if text.startswith("```"):
                    text = text.split("```")[1]
                    if text.startswith("json"):
                        text = text[4:]
                interpreted = json.loads(text)
                claude_map = {i["ticker"]: i for i in interpreted}
                print(f"Claude interpreted {len(interpreted)} trade(s).")
            except Exception as e:
                print(f"Claude interpretation failed: {e}")
                print("Falling back to keyword parsing.")
        else:
            print("No ANTHROPIC_API_KEY set. Using keyword parsing.")

    # Build final trade list
    for t in trades:
        instr = t["instruction"].upper()
        if instr == "SKIP" or instr == "":
            continue
        if instr == "APPROVE":
            result.append({
                "ticker": t["ticker"], "action": t["action"],
                "order_type": "MARKET", "shares": t["shares"],
                "limit_price": None, "stop_price": None,
                "note": "Approved as-is",
            })
        elif t["ticker"] in claude_map:
            ci = claude_map[t["ticker"]]
            if ci["action"] != "SKIP":
                result.append(ci)
            else:
                print(f"  Skipping {t['ticker']}: {ci.get('note', '')}")
        else:
            # Basic keyword fallback
            import re
            il = t["instruction"].lower()
            if "reduce" in il:
                m = re.search(r"(\d+)\s*shares?", il)
                if m:
                    result.append({
                        "ticker": t["ticker"], "action": t["action"],
                        "order_type": "MARKET", "shares": int(m.group(1)),
                        "limit_price": None, "stop_price": None,
                        "note": f"Reduced to {m.group(1)} shares",
                    })
                    continue
            if "limit" in il:
                m = re.search(r"\$?([\d.]+)", il)
                if m:
                    result.append({
                        "ticker": t["ticker"], "action": t["action"],
                        "order_type": "LIMIT", "shares": t["shares"],
                        "limit_price": float(m.group(1)),
                        "stop_price": None,
                        "note": f"Limit @ ${m.group(1)}",
                    })
                    continue
            print(f"  Cannot parse '{t['instruction']}' for {t['ticker']} - skipping")

    return result

final_trades = interpret_trades(trades)

if final_trades:
    display(Markdown(f"### Execution Plan ({len(final_trades)} trades)"))
    display(pd.DataFrame(final_trades))
else:
    print("No trades to execute after interpretation.")

---
## Step 6: Execute Trades

**This places REAL orders on your IB account.**

Set `CONFIRM = True` below and run to execute.

In [ ]:
# ============================================================
# SET TO True TO EXECUTE TRADES
# ============================================================
CONFIRM = False
# ============================================================

if not CONFIRM:
    print("Execution not confirmed.")
    print("Set CONFIRM = True above and re-run to execute.")
elif not final_trades:
    print("No trades to execute.")
else:
    # Reconnect in read-write mode
    ib.disconnect()
    ib_rw = IB()
    ib_rw.connect(
        IB_HOST, IB_PORT,
        clientId=IB_CLIENT_ID + 10,  # Different client ID
        readonly=False, timeout=10,
    )
    print(f"Connected for trading: {ib_rw.managedAccounts()[0]}\n")

    results = []
    for t in final_trades:
        if t["action"] == "SKIP":
            continue

        ticker = t["ticker"]
        action = t["action"]
        shares = t["shares"]
        order_type = t.get("order_type", "MARKET")
        ib_action = "BUY" if action in ("BUY", "BUY_TO_COVER") else "SELL"

        contract = Stock(ticker, "SMART", "USD")
        try:
            ib_rw.qualifyContracts(contract)
        except Exception as e:
            print(f"  {ticker}: FAILED to qualify - {e}")
            results.append({"ticker": ticker, "status": "FAILED", "message": str(e)})
            continue

        if order_type == "LIMIT" and t.get("limit_price"):
            order = LimitOrder(ib_action, shares, t["limit_price"])
        else:
            order = MarketOrder(ib_action, shares)

        print(f"  {ib_action} {shares} {ticker} ({order_type})...", end="")
        trade_obj = ib_rw.placeOrder(contract, order)
        ib_rw.sleep(2)

        status = trade_obj.orderStatus.status
        fill = trade_obj.orderStatus.avgFillPrice
        print(f" {status} (fill: ${fill:.2f})" if fill else f" {status}")

        results.append({
            "ticker": ticker, "status": status,
            "order_id": trade_obj.order.orderId,
            "fill_price": fill,
        })

        # Place trailing stop for BUY fills
        if (action == "BUY"
                and status in ("Filled", "Submitted", "PreSubmitted")
                and fill and fill > 0):
            ts_order = Order()
            ts_order.action = "SELL"
            ts_order.totalQuantity = shares
            ts_order.orderType = "TRAIL"
            ts_order.trailingPercent = TRAILING_STOP_PCT
            ts_order.tif = "GTC"
            ts_trade = ib_rw.placeOrder(contract, ts_order)
            ib_rw.sleep(1)
            init_stop = fill * (1 - TRAILING_STOP_PCT / 100)
            print(f"    Trailing stop {TRAILING_STOP_PCT}% "
                  f"(~${init_stop:.2f}): "
                  f"{ts_trade.orderStatus.status}")

    # Log results
    log_file = LIVE_DIR / "execution_log.csv"
    file_exists = log_file.exists()
    log_ts = datetime.now().isoformat()
    with open(log_file, "a", newline="") as f:
        w = csv.writer(f)
        if not file_exists:
            w.writerow(["timestamp", "ticker", "status",
                        "order_id", "fill_price"])
        for r in results:
            w.writerow([log_ts, r.get("ticker"), r.get("status"),
                        r.get("order_id", ""), r.get("fill_price", "")])

    # Summary
    filled = sum(1 for r in results if r["status"] in ("Filled", "Submitted", "PreSubmitted"))
    failed = sum(1 for r in results if r["status"] in ("ERROR", "FAILED"))
    display(Markdown(f"### Execution Summary"))
    print(f"Executed: {filled}  |  Failed: {failed}  |  Total: {len(results)}")
    display(pd.DataFrame(results))

    ib_rw.disconnect()
    print("\nDisconnected.")

---
## Cleanup

In [ ]:
if ib.isConnected():
    ib.disconnect()
    print("Disconnected from IB Gateway.")
else:
    print("Already disconnected.")